PROBLEM STATEMENT: 

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler

In [13]:
train=pd.read_csv('Train.csv')
test=pd.read_csv('Test.csv')

In [15]:
train.head()

,grade_A_Component_1,grade_A_Component_2,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area,x_component_1,x_component_2,x_component_3,x_component_4,x_component_5,class
0,0,1,150,36,144,172,947225,947332,439,439.099273,0,0,1,0,0,1
1,1,0,134,55,1144,1152,2379058,2379624,329,329.205616,1,0,0,0,0,1
2,1,0,135,65,950,974,1038442,1036754,300,300.120598,0,0,0,0,0,2
3,0,1,123,35,41,220,1705580,1705604,6803,6803.778622,0,0,1,0,0,1
4,1,0,138,69,462,466,1088124,1086579,251,251.401943,0,0,0,0,0,2


In [14]:
test.head()

,grade_A_Component_1,grade_A_Component_2,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area,x_component_1,x_component_2,x_component_3,x_component_4,x_component_5
0,1,0,123,65,33,54,1646439,1646893,632,632.391749,0,1,0,0,0
1,0,1,158,99,125,132,189874,189529,421,421.928609,0,0,0,0,0
2,0,1,120,35,64,75,12986873,12986862,272,272.212214,1,0,0,0,0
3,0,1,130,39,57,177,309634,310824,3312,3312.310581,0,0,1,0,0
4,1,0,142,75,23,79,5368307,5367467,862,862.499179,0,0,0,0,0


In [20]:
numeric_train=train.iloc[:,2:10]
numeric_test=test.iloc[:,2:10]

Very basic data preprocessing and fitting a model:

In [22]:
# Define the scaler 
SS = StandardScaler().fit(numeric_train)
columns=numeric_train.columns
# Scale the train set
numeric_train = SS.transform(numeric_train)
numeric_train = pd.DataFrame(numeric_train,columns=columns)
# Scale the test set
numeric_test = SS.transform(numeric_test)
numeric_test = pd.DataFrame(numeric_test,columns=columns)

In [23]:
numeric_train.head()

,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area
0,0.591354,-0.777115,-0.811333,-0.883497,-0.401060,-0.401011,-0.381579,-0.381681
1,-0.190164,-0.433562,1.105034,1.075244,0.404473,0.404769,-0.410242,-0.410316
2,-0.141319,-0.252745,0.733259,0.719473,-0.349742,-0.350704,-0.417798,-0.417895
3,-0.727458,-0.795196,-1.008719,-0.787559,0.025582,0.025578,1.276687,1.276759
4,0.005215,-0.180418,-0.201928,-0.295875,-0.321791,-0.322674,-0.430566,-0.430589


In [28]:
train_ss=pd.concat([numeric_train,train.drop(columns,axis=1)],axis=1)
test_ss=pd.concat([numeric_test,test.drop(columns,axis=1)],axis=1)

In [32]:
X=train_ss.drop('class',axis=1)
Y=train_ss['class']

In [42]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=400)

In [43]:
rfc.fit(X,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [44]:
predictions = rfc.predict_proba(test_ss)
print(predictions)

[[0.995  0.005 ]
 [0.1675 0.8325]
 [0.8675 0.1325]
 ...
 [0.9925 0.0075]
 [0.695  0.305 ]
 [0.975  0.025 ]]


In [46]:
submit = pd.DataFrame(predictions,columns=[1,2])
submit.to_csv('submit_1.csv')

Log loss score after first submission: 0.2961